# Load data from SQL, parse it appropriately

This script loads the data from a MIMIC-III database and parses the data for concepts required for the GOSISS project. The script outputs the `mimic-iii-gosiss-data.csv` file for later use.

In [ ]:
from __future__ import print_function

import psycopg2
import numpy as np
import pandas as pd
import os 

# cursors need to be rolled back if they fail
def execute_query_safely(sql, con):
    cur = con.cursor()
    
    # try to execute the query
    try:
        cur.execute(sql)
    except:
        # if an exception, rollback, rethrow the exception - finally closes the connection
        cur.execute('rollback;')
        raise
    finally:
        cur.close()
    
    return

import getpass

import matplotlib.pyplot as plt

%matplotlib inline

## Connect to database

In [ ]:
# prompt user for username/password
host='polya'
port=5432
print('Connecting to {} on port {} ...'.format(host,port))
sqluser = getpass.getuser()
sqlpass = getpass.getpass(prompt='Username: {}\nPassword: '.format(sqluser))

con = psycopg2.connect(dbname='mimic', host=host, port=port, user=sqluser, password=sqlpass)

## Create initial cohort

In [ ]:
# default is to write to public and read from both public and mimiciii
query_schema = "set search_path to public,mimiciii;"

# read in file/create base cohort
f = 'sql/cohort.sql'
with open(f, 'r') as fp:
    query = ''.join(fp.readlines())

# Execute the query
print('Generating materialized view using {} ...'.format(f),end=' ')
execute_query_safely(query_schema + query, con)
print('done.')

In [ ]:
# take a peek at the base cohort
query = query_schema + """select * from gosiss_cohort"""
co = pd.read_sql_query(query,con)

# print out the exclusions
print('Cohort - initial size: {} ICU stays'.format(co.shape[0]))
idxRem = np.zeros(co.shape[0])
for c in co.columns:
    if c.startswith('exclusion_'):
        print('  {:5g} ({:2.2f}%) - {}'.format(np.sum(co[c]),np.mean(co[c])*100.0, c))
        idxRem[co[c].values==1] = 1
        
print('Final cohort size: {} ICU stays ({:2.2f}%).'.format(co.shape[0] - np.sum(idxRem), (1-np.mean(idxRem))*100.0))

## Create necessary materialized views

In [ ]:
# get a list of all SQL files in the subfolder
query_path = './sql/'
queries = [f for f in os.listdir(query_path) 
             # only keep the filename if it is actually a file (and not a directory)
            if os.path.isfile(os.path.join(query_path,f))
             # and only keep the filename if it is an SQL file
            & f.endswith('.sql')
            # and we do *not* want the cohort - it's generated above
            & (f != 'cohort.sql')]

# make sure 'apsiii.sql' is the last query run
if 'apsiii.sql' in queries:
    queries.remove('apsiii.sql')
    queries.append('apsiii.sql')

# execute each SQL file to generate the materialized views
for f in queries:
    print('Executing {} ...'.format(f), end=' ')
    
    with open(os.path.join(query_path,f)) as fp:
        query = ''.join(fp.readlines())
        
    execute_query_safely(query_schema + query, con)
        
    print('done.')

# 2 - Extract all covariates and outcome measures

We now aggregate all the data from the various views into a single dataframe.

In [ ]:
# Load in the query from file
query = query_schema + """
--FINAL QUERY
select
  apsiii.*
from gosiss_cohort co
-- The following tables are generated by code within this repository
left join gosiss_apsiii apsiii
  on co.icustay_id = apsiii.icustay_id
-- apply exclusions
where co.exclusion_readmission = 0
  and co.exclusion_age = 0
  and co.exclusion_surgical = 0
  and co.exclusion_nodata = 0
  and co.exclusion_dnr = 0
order by co.icustay_id
"""

# Load the result of the query into a dataframe
df = pd.read_sql_query(query, con)
print('{} patients and {} features'.format(df.shape[0],df.shape[1]-2))

# 3 - Output the data to a csv file

In [ ]:
df.to_csv('mimic-iii-gosiss-data.csv',index=False)